# Lisp Metacircular Interpreter

In [1]:
(require '[clojure.test :refer [is]])

nil

## Closure Data Structure

In [2]:
(import 'clojure.lang.IFn)

(declare $eval)

(defrecord Closure [env params body]
  
  IFn
  
  (applyTo [self args]
    (let [extended-env (merge @env (zipmap params args))]
      ($eval body extended-env))))

(defn make-closure
  [env params body]
  (->Closure (atom env) params body))

(defn set-closure-env!
  [label-name closure]
  (swap! (:env closure)
         #(assoc % label-name closure)))

#'user/set-closure-env!

## Main Interpreter Function

### Special Forms in our Lisp Interpreter

- QUOTE: `(QUOTE datum)`
- IF: `(IF condition then-part else-part)`
- LAMBDA: `(LAMBDA (param1 param2 ...) body)`
- LABEL: `(LABEL name expr)`

In [3]:
(defn $eval
  [expr env]
  
  (cond
    
    ; Check for variable reference
    (symbol? expr)
    (if (contains? env expr)
      (env expr)
      (throw (RuntimeException. (str "Unbound variable: " expr))))
    
    ; Check for special forms
    (list? expr)
    (case (first expr)
      
      nil
      ()
      
      QUOTE
      (second expr)
      
      IF
      (let [[_ condition then-part else-part] expr]
        (if ($eval condition env)
          ($eval then-part env)
          ($eval else-part env)))
      
      LAMBDA
      (let [[_ params body] expr]
        (make-closure env params body))
      
      LABEL
      (let [[_ label-name label-expr] expr
            closure ($eval label-expr env)]
        (set-closure-env! label-name closure)
        closure)
      
      ; Ordinary function application
      (let [[fun & args] (map #($eval % env) expr)]
        (apply fun args)))
    
    ; Anything that is not symbol or a list evals to itself
    :else
    expr))

#'user/$eval

In [4]:
(is (= 42 ($eval 'X '{W 5, X 42})))
(try
  ($eval 'Y '{W 5, X 42})
  (is false)
  (catch RuntimeException e (is true)))
(is (= 42 ($eval 42 {})))
(is (= true ($eval true ($eval true {}))))
(is (= "hello" ($eval "hello" ($eval true {}))))

true

In [5]:
(is (= () ($eval () {})))
(is (= '(1 2 3) ($eval '(QUOTE (1 2 3)) {})))
(is (= 'X ($eval '(QUOTE X) {})))

true

In [6]:
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF false 2 3) {})))
(is (= 15 ($eval '(IF A B C) '{A 7, B 15, C 42})))
(is (= 42 ($eval '(IF A B C) '{A false, B 15, C 42})))

true

In [7]:
(is (= 6 ($eval '(PLUS 1 2 3) {'PLUS +})))
(is (= 28 ($eval '(TIMES (PLUS 2 2) (MINUS 10 3)) {'PLUS +, 'TIMES *, 'MINUS -})))
(is (= 28 ($eval '(* (+ 2 2) (- 10 3)) {'+ +, '* *, '- -})))
(is (= 0 ($eval '(+) {'+ +})))
(is (= '(nil) ($eval '(f nil) {'f list})))

true

In [8]:
(is (= 6 ($eval '((LAMBDA (X) (+ X 1)) 5) {'+ +})))
(is (= 15($eval '((LAMBDA (X Y)
                    (* X (+ Y 1)))
                  3
                  4)
                {'+ +, '* *})))
(is (= 42 ($eval '((LAMBDA () 42)) {})))
(is (= 8 ($eval '(((LAMBDA (X)
                     (LAMBDA (Y) 
                       (+ X Y)))
                   5)
                  3)
                {'+ +})))
(is (= 8 ($eval '((LAMBDA (F X) 
                    (F (F (F X))))
                  (LAMBDA (X) (* X 2))
                  1)
                {'* *})))

true

In [9]:
(is (= '(A A B B C C D D) 
       ($eval '((LABEL DUP (LAMBDA (X)
                             (IF (NULL? X)
                               ()
                               (CONS (CAR X)
                                     (CONS (CAR X)
                                           (DUP (CDR X)))))))
                (QUOTE (A B C D)))
              {'NULL? empty?
               'CONS cons
               'CAR first
               'CDR rest})))
(is (= 120 ($eval '((LABEL FACT (LAMBDA (N)
                                  (IF (ZERO? N)
                                    1
                                    (* N (FACT (- N 1))))))
                    5)
                  {'* *
                   'ZERO? zero?
                   '- -})))

true